In [1]:
import pandas as pd
import numpy as np

### This csv file contains Score, Summary and Text as preprocessed data

In [2]:
reviews_sst = pd.read_csv('preprocessed_reviews_SST.csv')

In [3]:
reviews_sst[:3]

,Score,Summary,Text
0,5,perfect dark chocol experi,well dark chocol creami lavend absolut perfect...
1,5,fresh qualiti,swear cupcak ate fresh one local groceri store...
2,5,wrong,origin given 1 star sinc cake best shape got s...


### split the data into train and test

In [4]:
rows = reviews_sst.shape[0]
rows

568454

In [5]:
train_size = int(rows*0.70)
test_size = rows - train_size

print(train_size)
print(test_size)

397917
170537


In [6]:
# creating new dataframes for train aand test data
train_data = pd.DataFrame(columns=reviews_sst.columns)
test_data  = pd.DataFrame(columns=reviews_sst.columns)

In [7]:
print("Train Data : {}".format(train_data))
print("\nTest Data: {}".format(test_data))
print("\ntest_size : {} train_size : {}".format(test_size, train_size))

Train Data : Empty DataFrame
Columns: [Score, Summary, Text]
Index: []

Test Data: Empty DataFrame
Columns: [Score, Summary, Text]
Index: []

test_size : 170537 train_size : 397917


In [8]:
import random

# indices list contains all indices from 0 to rows..
indices =list(range(rows))
random.shuffle(indices)

# get random indices from shuffled indices array for train and test data..
trnind = indices[:train_size]
tstind = indices[train_size:rows]

print(trnind[:10])
print(tstind[:10])

[156487, 293844, 61513, 45757, 178260, 6156, 218695, 275897, 381294, 77095]
[44856, 280988, 309808, 135195, 15138, 62155, 79897, 538523, 159837, 398555]


In [9]:
# split the data into train and test based on the indices that we saperated
train_data = reviews_sst.iloc[trnind]
test_data = reviews_sst.iloc[tstind]

In [10]:
train_data[:1]

,Score,Summary,Text
156487,5,excel snack bar,tast first fiber one bar thought run head eart...


In [11]:
test_data[:1]

,Score,Summary,Text
44856,5,like creami good get product,see food product explain anyon tast obvious wo...


In [12]:
# Counter is subclass of dict
from collections import Counter
# we will update this counter whenever needed
counter = Counter()

In [13]:
counter.clear()

In [14]:
# get summary and text from train data
summary = train_data['Summary']
print(summary.shape)
text = train_data['Text']
print(text.shape)

(397917,)
(397917,)


In [15]:
# get word count from summaries of all train data and update to the counter
for line in summary.iteritems():
    counter.update(str(line[1]).split())

In [16]:
# get word count from text data of train data and update the counter
for line in text.iteritems():
    counter.update(str(line[1]).split())

In [17]:
len(counter)

73500

In [18]:
import sys
sys.getsizeof(counter)

2621560

In [19]:
counter['best']

77092

In [20]:
counter.most_common(2)

[('like', 212309), ('tast', 207138)]

In [21]:
# reviews_sst.shape[0]
rows

568454

In [22]:
score_counts = [sum(reviews_sst['Score'] == 1), \
                sum(reviews_sst['Score'] == 2), \
                sum(reviews_sst['Score'] == 3), \
                sum(reviews_sst['Score'] == 4), \
                sum(reviews_sst['Score'] == 5)]
scores = dict(zip(list('12345'), score_counts))
print(sum(score_counts) == rows)
print(scores)

True
{'1': 52268, '2': 29769, '3': 42640, '4': 80655, '5': 363122}


In [23]:
# how many reviews are there that has rating
scores['1'], scores['2'], scores['3'], scores['4'], scores['5']

(52268, 29769, 42640, 80655, 363122)

In [24]:
word_count = dict()

In [25]:
for tple in train_data.itertuples():
    score = str(tple[1])
    summary = str(tple[2])
    text = str(tple[3])
    words = set(text.split())
    words.update(summary.split())
    for word in words:
        if word in word_count:
            word_count[word].update(score)
        else:
            word_count[word] = Counter(score)

# the following code should be used if you want to maintain word frequecy instead of word occurance in a document.

# for word in summary.split():
#     if word in word_count:
#         word_count[word].update(score)
#     else:
#         word_count[word] = Counter(score)
# for word in text.split():
#     if word in word_count:
#         word_count[word].update(score)
#     else:
#         word_count[word] = Counter(score)
        

In [26]:
len(word_count)

73500

In [27]:
#   word_count represents how many reviews(of specific score) that a perticular word occured. 
# It did't consider the case that the word occurred more than once in a single review. 


print(word_count['stick'])
print(word_count['great'])

Counter({'5': 3998, '4': 1470, '3': 1295, '1': 1258, '2': 1018})
Counter({'5': 94356, '4': 16732, '3': 5759, '1': 3480, '2': 2971})


# Testing Naive Bayes (Bernoulli model)  with our test data

In [28]:
#sample_test = test_data[:100]

In [29]:
test_data.head()

,Score,Summary,Text
44856,5,like creami good get product,see food product explain anyon tast obvious wo...
280988,5,good,great soft tasti better expect perfect loaf ev...
309808,5,perfect,fun littl gift share sunshin co worker friend ...
135195,5,great natur dog food,shelti love food 6 year old shelti allergi fee...
15138,5,hot cocoa cup,great price 24 count dad love gift sinc love h...


# Things that we will do to test our model

 1. We will consider scores (1,2) as NEGATIVE and (4,5) as POSITIVE and we will ignore reviews that has score 3.
 2. ie., we will consider scores(1,2,4,5) while building Confusion matrix.

___TP : (true positives)___ : # of reviews that are predicted POSITIVE and it is TRUE.
 
        ie., # of POSITIVE reviews that are predicted correctly (as POSITIVE)
___FP : (false positives)___ : # of reviews that are predicted as POSITIVE and it is FALSE.__(error)__
    
         ie., # of NEGATIVE reviews that are predicted incorrectly ( as POSITIVE)
___FN : (flase negatives)___ : # of reviews that are predicted as NEGATIVE and it is FALSE.__(error)__
 
         ie., # of POSITIVE reviews that are predicted incorrectly ( as NEGATIVE)
___TN : (true negative)___ : # of revies that are predicted as NEGATIVE and it is TRUE.

        ie., # of NEGATIVE reviews that are predicted correctly(as NEGATIVE)

### PRECISION

> ___  Out of all those reviews that you are predicted as +ve, HOW PRECISE ARE YOU ..?___

+ #of reviews that our model predicted as +ve = reviews that are (correctly predicted as +Ve) + ( incorrectly predicted as  +Ve)
+ #of times you correctly predicted as +Ve = TPs

$precision = \frac{TPs} {TPs+FPs}$


### RECALL

> ___Out of all those +Ve reviews, how many times you recalled(predicted) them correcly (positive).___ 

+ #of reviews that are +ve = +Ve reviews that are (correctly predicted as +Ve) + ( incorrectly predicted as  -Ve)
+ #of times you correctly predicted as +Ve = TPs

$recall = \frac{TPs} {TPs + FNs}$ 

In [30]:
# function to get probability
def prob(word, score):
    return (word_count[word][score] if word in word_count else 0 + 1)/(scores[score] + 2)
prob('like','4')

0.29931686028491017

> posterior $ \alpha$ (prior $\times$ likelihood)

+ but, since our data is imbalanced (80% are positive reviews), we will make prior as constant(0.5). ie., we are basically ignore that prior term.

> posterior $ \alpha $ likelihood

In [31]:
def likelihood(words, score):
    likelihood = 1
    for word in words:
        likelihood = likelihood * prob(word, score)
    return likelihood

In [32]:
def update_confusion_matrix(predicted, actual):
#     print(predicted, actual)
    # change this score into positive and negative
    predicted = 'positive' if 4<=predicted<=5 else 'negative'
    actual = 'positive' if 4<=actual<=5 else 'negative'

#     print(predicted, actual)
    if predicted is actual and actual is 'positive':
        # update true positives
        confusion_matrix.update({'TP':1})
    elif predicted is actual and actual is 'negative':
        # update true negatives
        confusion_matrix.update({'TN':1})
    elif predicted is not actual and actual is 'positive':
        # update false negatives
        confusion_matrix.update({'FN':1})
    elif predicted is not actual and actual is 'negative':
        # update false positives
        confusion_matrix.update({'FP':1})

In [33]:
# intialize confusion matrix
confusion_matrix = Counter()

for row in test_data.itertuples():
    score, summery, text = row[1:]
    # just getting all words at one place ( in a set )
    words = set(str(text).split())
    words.update(str(summery).split())
    # get all likelihoods of a review having some score into this list
    probs = list()
    for rating in list('12345'):
        probs.append(likelihood(words, rating))
    # store our predicted score into pred_score variable
    pred_score = np.argmax(probs)+1 
    
    # update the confusion matrix for reviews that are not having score as 3.
    if score != 3:
        update_confusion_matrix(pred_score, score)
    

## Confusion Matrix

In [34]:
confusion_matrix

Counter({'FN': 29303, 'FP': 2757, 'TN': 22049, 'TP': 103628})

In [35]:
recall = confusion_matrix['TP']/(confusion_matrix['TP'] + confusion_matrix['FN'])
recall

0.7795623293287495

##  PRECISION

$ precision  = \frac{TP} {TP + FP} $

In [36]:
precision = confusion_matrix['TP']/(confusion_matrix['TP'] + confusion_matrix['FP'])
print('Precision: {}'.format(precision))

Precision: 0.9740846923908446


## RECALL 

$ recall = \frac{TP}{TP + FN}$

In [37]:
recall = confusion_matrix['TP']/(confusion_matrix['TP']+confusion_matrix['FN'])
print('recall: {}'.format(recall))

recall: 0.7795623293287495


## Accuracy of our model (F score as metric)

+ We use $F_1$ score or simply F-score as our accuracy with testing data
+ basically $F_1$ score is _harmonic mean_ of _precision_ and _recall_

$F_1 = 2 . \frac{precision . recall}{precision + recall}$

In [38]:
f_score = 2*(precision*recall)/(precision+recall)
print('F score : {}'.format(f_score))

F score : 0.866034866034866
